In [1]:
import pandas as pd
import numpy as np

import re
from itertools import chain
from collections import Counter

from google.cloud import bigquery
from google.oauth2 import service_account

from datetime import datetime, timedelta

import requests
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning

# InsecureRequestWarning 경고 무시
warnings.simplefilter('ignore', InsecureRequestWarning)

In [2]:
KEY_PATH = ".config/"
servicekey_path = KEY_PATH + "serviceKey.json" ## 빅쿼리 외 다른 API 활용 위해
bigquerykey_path = KEY_PATH + "mido-project-426906-31b49963ac97.json"

warnings.filterwarnings("ignore")

In [3]:
# BigQuery 클라이언트 생성 함수
def create_bigquery_client(key_path):
    credentials = service_account.Credentials.from_service_account_file(key_path)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
    return client

In [4]:
def save_dataframe_to_bigquery(df, dataset_id, table_id, key_path):
    # BigQuery 클라이언트 객체 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 데이터프레임을 BigQuery 테이블에 적재
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = "WRITE_TRUNCATE"  # 기존 테이블 내용 삭제 후 삽입

    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # 작업 완료 대기

    print(f"Data inserted into table {table_id} successfully.")

In [5]:
def get_dataframe_from_bigquery(dataset_id, table_id, key_path):
    # BigQuery 클라이언트 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 테이블 데이터를 DataFrame으로 변환
    df = client.list_rows(table_ref).to_dataframe()

    return df

In [6]:
# 오늘 날짜
today = datetime.today()#.strftime('%Y%m%d')

# 어제 날짜 계산
ytday = datetime.today() - timedelta(days=1)

# 만약 어제, 오늘이 토요일(5) 또는 일요일(6)이라면, 그 전주 금요일로 변경
if ytday.weekday() == 5:  # 토요일
    ytday -= timedelta(days=1)
elif ytday.weekday() == 6:  # 일요일
    ytday -= timedelta(days=2)
if today.weekday() == 5:  # 토요일
    today -= timedelta(days=1)
elif today.weekday() == 6:  # 일요일
    today -= timedelta(days=2)

# 'YYYYMMDD' 형식으로 변환
ytday = ytday.strftime('%Y%m%d')
today = today.strftime('%Y%m%d')

#### 법정동코드

In [7]:
dist_code = pd.read_csv('C:\py_src\midoproject\data/법정동코드 전체자료.csv',encoding='cp949')

In [8]:
dist_code_list = []
for i in range(dist_code['법정동명'].str.split().apply(len).max()):
    dist_code_temp = dist_code[dist_code['법정동명'].str.split().str[i].str[-1].isin(['읍','면','동','리','가','로']).dropna()]
    dist_code_list.append([dist_code_temp['법정동명'].str.split().str[i].dropna().unique()])

emd_code_list = list(chain(*chain(*dist_code_list)))

#### 종합쇼핑몰 납품상세내역

In [9]:
all_shop_df = get_dataframe_from_bigquery('g2b', 'shop_detail_df_all', bigquerykey_path)

In [10]:
all_shop_df_fin = all_shop_df[['납품요구접수일자', '수요기관명', '납품요구건명', '업체명', '단가', '단위', '수량', '금액', '수요기관코드', '수요기관구분', '수요기관지역명','납품요구지청명']]
all_shop_df_fin = all_shop_df_fin.sort_values(['납품요구접수일자'],ascending=False).reset_index(drop=True)

In [11]:
# 특수문자, 숫자, 영어 제거 함수 (연속된 공백을 단일 공백으로 변환 포함)
def clean_text(text):
    text = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', ' ', text)
    text = re.sub('\s+', ' ', text).strip()  # 연속된 공백을 단일 공백으로 변환
    return text

In [12]:
# 사업명 전처리
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명'].apply(clean_text)
# all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.strip()

all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.split().apply(lambda x: [kw for kw in x if len(kw) > 1]) ## 글자수 1 제외
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].apply((' ').join)

In [14]:
# 수요기관 필터링
dist_nm = pd.Series(all_shop_df_fin[all_shop_df_fin['수요기관지역명'].str.split(' ').str[1].notnull()]['수요기관지역명'].str.split(' ').str[1].unique())
dist_nm = np.where(dist_nm.apply(len)<=2,dist_nm,dist_nm.str[:-1])

In [15]:
# 전체 키워드 데이터셋
mapping_keywd_all = list(chain(*all_shop_df_fin['납품요구건명_re'].str.split(' '))) ## 전체 키워드
mapping_keywd_df = pd.DataFrame(Counter(mapping_keywd_all).items(),columns=['키워드','빈도수'])
mapping_keywd_df = mapping_keywd_df[mapping_keywd_df['키워드'].apply(len)!=1].reset_index(drop=True)
mapping_keywd_df.sort_values('빈도수',ascending=False)

,키워드,빈도수
1,인조잔디,861
3,관급자재,483
19,구입,268
26,구매,152
24,운동장,108
...,...,...
606,체육시설탄성포장재,1
610,중대재해,1
611,예방,1
613,대산읍,1


In [17]:
# 종합쇼핑몰 기준 중요키워드 추출
print('1등급 키워드 : ', list(mapping_keywd_df[mapping_keywd_df['빈도수'] >= 50].sort_values('빈도수',ascending=False)['키워드']))
print('2등급 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 50) & (mapping_keywd_df['빈도수'] >= 40)].sort_values('빈도수',ascending=False)['키워드']))
print('3등급 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 40) & (mapping_keywd_df['빈도수'] >= 30)].sort_values('빈도수',ascending=False)['키워드']))
print('4등급 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 30) & (mapping_keywd_df['빈도수'] >= 20)].sort_values('빈도수',ascending=False)['키워드']))
print('5등급 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 20) & (mapping_keywd_df['빈도수'] >= 10)].sort_values('빈도수',ascending=False)['키워드']))
print('기타 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 10)].sort_values('빈도수',ascending=False)['키워드']))

1등급 키워드 :  ['인조잔디', '관급자재', '구입', '구매', '운동장', '조성공사', '공사', '테니스장', '관급', '조성사업', '게이트볼장', '체육시설', '정비공사', '교체공사']
2등급 키워드 :  ['조성']
3등급 키워드 :  ['축구장', '설치', '풋살장', '개선공사']
4등급 키워드 :  ['족구장', '파크골프장', '설치공사', '교체', '보수공사', '토목', '조달구매', '환경개선']
5등급 키워드 :  ['환경개선공사', '개보수공사', '정비사업', '다목적구장', '시설개선사업', '개선사업', '건축', '시설개선공사', '시설개선', '잔디', '보수', '조달', '연병장', '부대', '개선', '단계', '생활체육공원', '사업', '다목적', '물품', '체육공원', '개소']
기타 키워드 :  ['추가', '교체사업', '조경', '어린이공원', '생활체육시설', '관련', '신축공사', '정비', '실내게이트볼장', '야구장', '개보수', '신축', '시설', '해운대수목원', '체육', '구장', '김영훈', '차분', '위한', '따른', '관급자', '노후시설', '요청', '창녕국민체육센터', '트랙조성', '배수판', '설치사업', '시행', '당진', '리모델링', '공공하수처리시설', '구입의뢰', '인조잔디운동장', '치수과', '증설공사', '본부', '그라운드골프장', '재정비공사', '건립', '건축공사', '환경개선사업', '확장', '여단', '하부', '사단', '비가림시설', '학교운동장', '근린공원', '조경공사', '대보수', '배드민턴장', '일원', '재정비사업', '관급자관급', '공공체육시설', '운동장환경개선', '시설물', '우수조달', '스포츠파크', '구매설치', '옥외', '자재', '휀스', '울타리', '인조', '그라운드', '본원', '수지아르피아', '환경친화적', '체육시설관리과', '수해복구공사', '다목적체육관', '트랙보수'

In [18]:
# 빈도수 키워드
freq_kwd = ('|').join(list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 10)]['키워드']))

#### 지자체 세부사업별 예산서

In [19]:
# 빅쿼리에서 불러오기
budget_df_today = get_dataframe_from_bigquery('budget','budget_df_0' + today,bigquerykey_path)
budget_df_ytday = get_dataframe_from_bigquery('budget','budget_df_0' + ytday,bigquerykey_path)

In [20]:
# 종료 사업
budget_df_delete_temp1 = budget_df_ytday[budget_df_ytday['세부사업코드'].isin(set(budget_df_ytday['세부사업코드']) - set(budget_df_today['세부사업코드']))].reset_index(drop=True)
budget_df_delete_temp2 = budget_df_ytday[budget_df_ytday['세부사업명'].isin(set(budget_df_ytday['세부사업명']) - set(budget_df_today['세부사업명']))].reset_index(drop=True)
budget_df_delete = pd.concat([budget_df_delete_temp1,budget_df_delete_temp2],axis=0).drop_duplicates().reset_index(drop=True)
budget_df_delete

,회계연도,지역코드,지역명,자치단체코드,자치단체명,회계구분명,세부사업코드,세부사업명,집행일자,예산현액,지출액,편성액,분야명,부문명,행정자치단체코드


In [21]:
# 새로 추가된 사업
budget_df_new_temp1 = budget_df_today[~budget_df_today['세부사업코드'].isin(budget_df_ytday['세부사업코드'])].reset_index(drop=True).reset_index(drop=True)
budget_df_new_temp2 = budget_df_today[~budget_df_today['세부사업명'].isin(budget_df_ytday['세부사업명'])].reset_index(drop=True).reset_index(drop=True)
budget_df_new = pd.concat([budget_df_new_temp1,budget_df_new_temp2],axis=0).drop_duplicates().reset_index(drop=True)
budget_df_new

,회계연도,지역코드,지역명,자치단체코드,자치단체명,회계구분명,세부사업코드,세부사업명,집행일자,예산현액,지출액,편성액,분야명,부문명,행정자치단체코드


In [23]:
# 필요키워드1
need_kwd1 = '학교|초등학교|중학교|고등학교|대학교|다목적|운동장|종합운동장|공원|체육|체육공원|체육시설|체육센터|스포츠|스포츠센터|놀이시설|연병장|축구장|족구장|야구장|풋살장|배드민턴장|테니스장|게이트볼장|하키장'

# 필요키워드2
need_kwd2 = '개선|교체|보수|공사|구매|구입|설치|정비|조성'

# 필요키워드3
need_kwd3 = '인조잔디'

# 제외키워드
except_kwd = 'CCTV|cctv|LED|led|가로등|가속기|간접등|간판|개방|검진|경관|계단|골프장|관리|광고|교량|교실|교육|교차로|균열|그늘막|그림|급수관|급식|난방기|내진|냉방기|네트워크|노상|노점|농기계|단열|데이터|도로개설|도로방음벽|도로정비|도로확장|도서관|디자인|뚜껑|막구조물|맨홀|모노레일|모니터링|미끄럼|미세먼지|미술|방송|방수|방음|방음벽|배면도로|배수|버스|법률자문|벤치|벽화|보건|보도블럭|보상|보일러|보행자도로|복지|볼라드|분묘|불법|빗물받이|사용료|사워실|살포|생육환경|석면|선홈통|소프트웨어|수도|수로|수목표찰|수영장|스크린|스프링클러|승강기|시계탑|시스템|식수대|신호등|안심벨|안전|약수터|억제제|역량|연결도로|예방|예술|오염|옹벽|요양기관|운영|울타리|위생|위험수목|유아|육교|음성안내|음악|의자|인양기|인터넷|입학|자료|자전거|저장고|저장장치|전광판|전기|전신주|전통시장|정화|정화조|조도|조명|조형물|조화기|주차장|줄눈|진로진학|진입도로|창고|창출|처우|철거|취업|치안|카메라|카약|통학로|파고라|펜스|폐기물|폐열|포충기|풀장|풍차|하수|하수관로|하수도|하천|학습|행사|화장|화장실|활성화|횡단보도|휀스|흡연'

In [24]:
budget_df_today = budget_df_today[((budget_df_today['세부사업명'].str.contains(need_kwd1) & 
                                  budget_df_today['세부사업명'].str.contains(need_kwd2)) &
                                  ~budget_df_today['세부사업명'].str.contains(except_kwd)) |
                                  budget_df_today['세부사업명'].str.contains(need_kwd3)
                                  ].reset_index(drop=True)

In [25]:
# 사업명 전처리
budget_df_today['세부사업명_re'] = budget_df_today['세부사업명'].apply(clean_text)
# budget_df_today['세부사업명_re'] = budget_df_today['세부사업명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
budget_df_today['세부사업명_re'] = budget_df_today['세부사업명_re'].str.strip()

budget_df_today['세부사업명_re'] = budget_df_today['세부사업명_re'].str.split().apply(lambda x: [kw for kw in x if len(kw) > 1]) ## 글자수 1 제외
budget_df_today['세부사업명_re'] = budget_df_today['세부사업명_re'].apply((' ').join)

In [26]:
# 해당 지역 추출
filtered_budget_df_temp1 = budget_df_today[budget_df_today['자치단체명'].str.contains(('|').join(dist_nm))].reset_index(drop=True)
filtered_budget_df_temp2 = budget_df_today[budget_df_today['자치단체명'].str.contains('본청')].reset_index(drop=True)

filtered_budget_df_today = pd.concat([filtered_budget_df_temp1,filtered_budget_df_temp2],axis=0).reset_index(drop=True)

#### 지자체 종합쇼핑몰 전처리

In [27]:
# Initialize the dictionary for split keywords
shop_split_kwd = {}
for i in range(all_shop_df_fin['납품요구건명_re'].str.split().apply(len).max()):
    shop_split_kwd['kwd_' + str(i)] = all_shop_df_fin['납품요구건명_re'].str.split().str[i]

In [28]:
# 매핑데이터셋
mapping_merge_df = []

In [29]:
# 종합쇼핑몰 납품요구건명 split 키워드 순서대로 맵핑
# Initialize the list to store the results
prep_budget_df_today = filtered_budget_df_today.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

# Iterate over each keyword column
for i in range(len(shop_split_kwd)):
    keyword_column = 'kwd_' + str(i)
    
    # Get unique non-null keywords
    shop_split_kwd_temp = shop_split_kwd[keyword_column].dropna().unique()

    prep_budget_df_today[keyword_column] = prep_budget_df_today['세부사업명_re'].apply(lambda x: ' '.join(re.findall('|'.join(shop_split_kwd_temp), x)))
    prep_budget_df_today[keyword_column] = prep_budget_df_today[keyword_column].str.strip()
    prep_budget_df_today[keyword_column] = prep_budget_df_today[keyword_column].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

    prep_all_shop_df_fin[keyword_column] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall('|'.join(shop_split_kwd_temp), x)))
    prep_all_shop_df_fin[keyword_column] = prep_all_shop_df_fin[keyword_column].str.strip()
    prep_all_shop_df_fin[keyword_column] = prep_all_shop_df_fin[keyword_column].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

    mapping_merge_df_temp = pd.merge(
            prep_budget_df_today[(prep_budget_df_today[keyword_column] != '') & ~(prep_budget_df_today[keyword_column].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', keyword_column]].reset_index(drop=True),
            prep_all_shop_df_fin[(prep_all_shop_df_fin[keyword_column] != '') & ~(prep_all_shop_df_fin[keyword_column].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', keyword_column]].reset_index(drop=True),
            how='left',
            on=keyword_column
            ).drop_duplicates().reset_index(drop=True)
    
    # Append the temporary DataFrame to the results list
    mapping_merge_df.append(mapping_merge_df_temp)

In [30]:
# 종합쇼핑몰 지역 키워드 맵핑 - 세부사업명 지역 및 해당지역 추출
# Initialize the columns for district keywords
prep_budget_df_today = filtered_budget_df_today.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

sig_pat = '|'.join(dist_nm)

prep_budget_df_today['kwd_sig'] = prep_budget_df_today['세부사업명_re'].apply(lambda x: ' '.join(re.findall(sig_pat, x)))
prep_budget_df_today['kwd_sig'] = prep_budget_df_today['kwd_sig'].str.strip()
prep_budget_df_today['kwd_sig'] = prep_budget_df_today['kwd_sig'].str.split()
prep_budget_df_today = prep_budget_df_today.explode('kwd_sig')

prep_all_shop_df_fin['kwd_sig'] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall(sig_pat, x)))
prep_all_shop_df_fin['kwd_sig'] = prep_all_shop_df_fin['kwd_sig'].str.strip()
prep_all_shop_df_fin['kwd_sig'] = prep_all_shop_df_fin['kwd_sig'].str.split()
prep_all_shop_df_fin = prep_all_shop_df_fin.explode('kwd_sig')

mapping_merge_df_sig = pd.merge(
    prep_budget_df_today[(prep_budget_df_today['kwd_sig'] != '') & ~(prep_budget_df_today['kwd_sig'].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', 'kwd_sig']].reset_index(drop=True),
    prep_all_shop_df_fin[(prep_all_shop_df_fin['kwd_sig'] != '') & ~(prep_all_shop_df_fin['kwd_sig'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_sig']].reset_index(drop=True),
    how='left',
    on='kwd_sig'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_sig)

In [31]:
# 종합쇼핑몰 지역 키워드 맵핑 - 세부사업명 지역 및 해당지역 읍면동 추출
# Initialize the columns for district keywords
prep_budget_df_today = filtered_budget_df_today.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

emd_pat = '|'.join(set(emd_code_list))

prep_budget_df_today['kwd_emd'] = prep_budget_df_today['세부사업명_re'].apply(lambda x: ' '.join(re.findall(emd_pat, x)))
prep_budget_df_today['kwd_emd'] = prep_budget_df_today['kwd_emd'].str.strip()
prep_budget_df_today['kwd_emd'] = prep_budget_df_today['kwd_emd'].str.split()
prep_budget_df_today = prep_budget_df_today.explode('kwd_emd')

prep_all_shop_df_fin['kwd_emd'] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall(emd_pat, x)))
prep_all_shop_df_fin['kwd_emd'] = prep_all_shop_df_fin['kwd_emd'].str.strip()
prep_all_shop_df_fin['kwd_emd'] = prep_all_shop_df_fin['kwd_emd'].str.split()
prep_all_shop_df_fin = prep_all_shop_df_fin.explode('kwd_emd')

mapping_merge_df_emd = pd.merge(
    prep_budget_df_today[(prep_budget_df_today['kwd_emd'] != '') & ~(prep_budget_df_today['kwd_emd'].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', 'kwd_emd']].reset_index(drop=True),
    prep_all_shop_df_fin[(prep_all_shop_df_fin['kwd_emd'] != '') & ~(prep_all_shop_df_fin['kwd_emd'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_emd']].reset_index(drop=True),
    how='left',
    on='kwd_emd'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_emd)

In [32]:
# 필요키워드 앞부분 추출
need_kwd_split = list(set(chain(*all_shop_df_fin['납품요구건명_re'].apply(lambda x: re.split(need_kwd1,x)).apply(''.join).str.split())))
need_kwd_split_list = list(pd.Series(need_kwd_split)[pd.Series(need_kwd_split).apply(len) >= 2])

In [33]:
# 종합쇼핑몰 지역 키워드 맵핑 - 필요키워드 앞부분 매핑
# Initialize the columns for district keywords
prep_budget_df_today = filtered_budget_df_today.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

kwd_split_pat = '|'.join(set(need_kwd_split_list))

prep_budget_df_today['kwd_need_split'] = prep_budget_df_today['세부사업명_re'].apply(lambda x: ' '.join(re.findall(kwd_split_pat, x)))
prep_budget_df_today['kwd_need_split'] = prep_budget_df_today['kwd_need_split'].str.strip()
prep_budget_df_today['kwd_need_split'] = prep_budget_df_today['kwd_need_split'].str.split()
prep_budget_df_today = prep_budget_df_today.explode('kwd_need_split')

prep_all_shop_df_fin['kwd_need_split'] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall(kwd_split_pat, x)))
prep_all_shop_df_fin['kwd_need_split'] = prep_all_shop_df_fin['kwd_need_split'].str.strip()
prep_all_shop_df_fin['kwd_need_split'] = prep_all_shop_df_fin['kwd_need_split'].str.split()
prep_all_shop_df_fin = prep_all_shop_df_fin.explode('kwd_need_split')

mapping_merge_df_need_split = pd.merge(
    prep_budget_df_today[(prep_budget_df_today['kwd_need_split'] != '') & ~(prep_budget_df_today['kwd_need_split'].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', 'kwd_need_split']].reset_index(drop=True),
    prep_all_shop_df_fin[(prep_all_shop_df_fin['kwd_need_split'] != '') & ~(prep_all_shop_df_fin['kwd_need_split'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_need_split']].reset_index(drop=True),
    how='left',
    on='kwd_need_split'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_need_split)

In [34]:
# 종합쇼핑몰 지역 키워드 맵핑 - 필요키워드 매핑
# Initialize the columns for district keywords
prep_budget_df_today = filtered_budget_df_today.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

prep_budget_df_today['kwd_need'] = prep_budget_df_today['세부사업명_re'].apply(lambda x: ' '.join(re.findall(need_kwd1, x)))
prep_budget_df_today['kwd_need'] = prep_budget_df_today['kwd_need'].str.strip()
prep_budget_df_today['kwd_need'] = prep_budget_df_today['kwd_need'].str.split()
prep_budget_df_today = prep_budget_df_today.explode('kwd_need')

prep_all_shop_df_fin['kwd_need'] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall(need_kwd1, x)))
prep_all_shop_df_fin['kwd_need'] = prep_all_shop_df_fin['kwd_need'].str.strip()
prep_all_shop_df_fin['kwd_need'] = prep_all_shop_df_fin['kwd_need'].str.split()
prep_all_shop_df_fin = prep_all_shop_df_fin.explode('kwd_need')

mapping_merge_df_need = pd.merge(
    prep_budget_df_today[(prep_budget_df_today['kwd_need'] != '') & ~(prep_budget_df_today['kwd_need'].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', 'kwd_need']].reset_index(drop=True),
    prep_all_shop_df_fin[(prep_all_shop_df_fin['kwd_need'] != '') & ~(prep_all_shop_df_fin['kwd_need'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_need']].reset_index(drop=True),
    how='left',
    on='kwd_need'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_need)

In [35]:
# 키워드별 데이터셋 병합
mapping_merge_df_fin = pd.concat(mapping_merge_df, axis=0).drop_duplicates()
mapping_merge_df_fin = mapping_merge_df_fin[~mapping_merge_df_fin['납품요구건명'].isnull()].reset_index(drop=True)

In [36]:
# 모든 키워드 합
kwd_columns = ['kwd_0','kwd_1','kwd_2','kwd_3','kwd_4','kwd_5','kwd_6','kwd_7','kwd_8','kwd_sig','kwd_emd','kwd_need_split','kwd_need']
mapping_merge_df_fin['all_kwd'] = mapping_merge_df_fin[kwd_columns].apply(lambda x : (' ').join(x.dropna()),axis=1)
mapping_merge_df_fin['all_kwd'] = mapping_merge_df_fin['all_kwd'].str.strip()
mapping_merge_df_fin['all_kwd'] = mapping_merge_df_fin['all_kwd'].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
mapping_merge_df_fin.drop(columns=kwd_columns, inplace=True)

In [37]:
# 지역명 추출1
mapping_merge_df_fin['수요기관지역명_split1'] = mapping_merge_df_fin['수요기관지역명'].str.split().str[0]

# Define the replacements
replacements = {
    '전라북도': '전북',
    '전라남도': '전남',
    '경상북도': '경북',
    '경상남도': '경남',
    '충청북도': '충북',
    '충청남도': '충남'
}

# Apply the replacements
mapping_merge_df_fin['수요기관지역명_split1'] = mapping_merge_df_fin['수요기관지역명_split1'].replace(replacements)
mapping_merge_df_fin['수요기관지역명_split1'] = mapping_merge_df_fin['수요기관지역명_split1'].str[:2]

In [38]:
# 지역명 추출2
mapping_merge_df_fin['수요기관지역명_split2'] = mapping_merge_df_fin['수요기관지역명'].str.split().str[1]
dist_sig_pat = '|'.join(mapping_merge_df_fin['수요기관지역명_split2'].dropna().unique())

mapping_merge_df_fin['자치단체명_re'] = mapping_merge_df_fin['자치단체명'].apply(lambda x: ' '.join(re.findall(dist_sig_pat, x)))
mapping_merge_df_fin['자치단체명_re'] = mapping_merge_df_fin['자치단체명_re'].str.strip()
mapping_merge_df_fin['자치단체명_re'] = mapping_merge_df_fin['자치단체명_re'].str.split()
mapping_merge_df_fin = mapping_merge_df_fin.explode('자치단체명_re')

In [39]:
# 제외 지역 제거
filtered_mapping_merge_df_fin = mapping_merge_df_fin[mapping_merge_df_fin['자치단체명_re']!=''].drop_duplicates().reset_index(drop=True)

In [40]:
# 지역 일치 확인1
mapping_merge_df_fin_temp1 = filtered_mapping_merge_df_fin[filtered_mapping_merge_df_fin['지역명']==filtered_mapping_merge_df_fin['수요기관지역명_split1']].drop_duplicates().reset_index(drop=True)
mapping_merge_df_fin_temp1 = mapping_merge_df_fin_temp1[mapping_merge_df_fin_temp1['자치단체명'].str.contains('|'.join(mapping_merge_df_fin_temp1['수요기관지역명_split2'].dropna().unique()))].drop_duplicates().reset_index(drop=True)
mapping_merge_df_fin_temp1 = mapping_merge_df_fin_temp1[mapping_merge_df_fin_temp1['자치단체명_re']==mapping_merge_df_fin_temp1['수요기관지역명_split2']].drop_duplicates().reset_index(drop=True)

# 본청 포함 확인
mapping_merge_df_fin_temp2 = filtered_mapping_merge_df_fin[filtered_mapping_merge_df_fin['자치단체명'].str.contains('본청')].drop_duplicates().reset_index(drop=True)

# 데이터 병합
mapping_merge_df_final = pd.concat([mapping_merge_df_fin_temp1,mapping_merge_df_fin_temp2], axis=0).drop_duplicates().reset_index(drop=True)

In [41]:
# 그룹화할 컬럼 리스트
group_cols = ['지역명', '자치단체명', '세부사업코드', '세부사업명', '세부사업명_re', '수요기관명', '수요기관지역명', '납품요구건명', '납품요구건명_re','수요기관지역명_split1', '자치단체명_re', '수요기관지역명_split2']

# 그룹화 및 all_kwd 컬럼 조인
mapping_merge_df_final_grp = mapping_merge_df_final.groupby(group_cols).agg({'all_kwd': ' '.join}).reset_index()
mapping_merge_df_final_grp['all_kwd'] = mapping_merge_df_final_grp['all_kwd'].str.strip()
mapping_merge_df_final_grp['all_kwd'] = mapping_merge_df_final_grp['all_kwd'].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

In [42]:
# 필요키워드 기준
mapping_merge_df_final_grp['세부사업명_키워드1'] = mapping_merge_df_final_grp['세부사업명'].apply(lambda x: ' '.join(re.findall(need_kwd1, x)))
mapping_merge_df_final_grp['납품요구건명_키워드1'] = mapping_merge_df_final_grp['납품요구건명'].apply(lambda x: ' '.join(re.findall(need_kwd1, x)))

# 사업명 지역 추출
mapping_merge_df_final_grp['세부사업명_지역'] = mapping_merge_df_final_grp['세부사업명'].apply(lambda x: ' '.join(re.findall(emd_pat, x)))
mapping_merge_df_final_grp['납품요구건명_지역'] = mapping_merge_df_final_grp['납품요구건명'].apply(lambda x: ' '.join(re.findall(emd_pat, x)))

# 필요키워드 패턴 기준
need_kwd_pat = '\w+' + ('|\w+').join(need_kwd1.split('|'))

mapping_merge_df_final_grp['세부사업명_키워드2'] = mapping_merge_df_final_grp['세부사업명'].apply(lambda x: ' '.join(re.findall(need_kwd_pat, x)))
mapping_merge_df_final_grp['납품요구건명_키워드2'] = mapping_merge_df_final_grp['납품요구건명'].apply(lambda x: ' '.join(re.findall(need_kwd_pat, x)))

In [43]:
def check_inclusion1(row):
    set1 = set(row['세부사업명_키워드1'])
    set2 = set(row['납품요구건명_키워드1'])
    return bool(set1 & set2)

In [44]:
# 필요키워드 모두 없는것 제외
mapping_merge_df_final_grp_temp1 = mapping_merge_df_final_grp[(mapping_merge_df_final_grp['세부사업명_키워드1']!='')&(mapping_merge_df_final_grp['납품요구건명_키워드1']!='')].reset_index(drop=True)

# 필요키워드 포함여부 확인
mapping_merge_df_final_grp_temp1['세부사업명_키워드1'] = mapping_merge_df_final_grp_temp1['세부사업명_키워드1'].str.split() ## 키워드별 정리
mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'] = mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'].str.split() ## 키워드별 정리

mapping_merge_df_final_grp_temp1 = mapping_merge_df_final_grp_temp1[mapping_merge_df_final_grp_temp1.apply(check_inclusion1,axis=1)].reset_index(drop=True)

mapping_merge_df_final_grp_temp1['세부사업명_키워드1'] = mapping_merge_df_final_grp_temp1['세부사업명_키워드1'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'] = mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

In [45]:
def check_inclusion_dist(row):
    set1 = set(row['세부사업명_지역'])
    set2 = set(row['납품요구건명_지역'])
    return bool(set1 & set2)

In [46]:
# 지역 없는것 제외
mapping_merge_df_final_grp_temp2 = mapping_merge_df_final_grp[(mapping_merge_df_final_grp['세부사업명_지역']!='')&(mapping_merge_df_final_grp['납품요구건명_지역']!='')].reset_index(drop=True)

# 사업명 지역 포함여부 확인
mapping_merge_df_final_grp_temp2['세부사업명_지역'] = mapping_merge_df_final_grp_temp2['세부사업명_지역'].str.split()
mapping_merge_df_final_grp_temp2['납품요구건명_지역'] = mapping_merge_df_final_grp_temp2['납품요구건명_지역'].str.split()

mapping_merge_df_final_grp_temp2 = mapping_merge_df_final_grp_temp2[mapping_merge_df_final_grp_temp2.apply(check_inclusion_dist,axis=1)].reset_index(drop=True)

mapping_merge_df_final_grp_temp2['세부사업명_지역'] = mapping_merge_df_final_grp_temp2['세부사업명_지역'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip() ## 키워드별 정리
mapping_merge_df_final_grp_temp2['납품요구건명_지역'] = mapping_merge_df_final_grp_temp2['납품요구건명_지역'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip() ## 키워드별 정리

In [47]:
def check_inclusion2(row):
    set1 = set(row['세부사업명_키워드2'])
    set2 = set(row['납품요구건명_키워드2'])
    return bool(set1 & set2)

In [48]:
# 필요키워드패턴 모두 없는것 제외
mapping_merge_df_final_grp_temp3 = mapping_merge_df_final_grp[(mapping_merge_df_final_grp['세부사업명_키워드2']!='')&(mapping_merge_df_final_grp['납품요구건명_키워드2']!='')].reset_index(drop=True)

# 필요키워드 패턴 포함여부 확인
mapping_merge_df_final_grp_temp3['세부사업명_키워드2'] = mapping_merge_df_final_grp_temp3['세부사업명_키워드2'].str.split() ## 키워드별 정리
mapping_merge_df_final_grp_temp3['납품요구건명_키워드2'] = mapping_merge_df_final_grp_temp3['납품요구건명_키워드2'].str.split() ## 키워드별 정리

mapping_merge_df_final_grp_temp3 = mapping_merge_df_final_grp_temp3[mapping_merge_df_final_grp_temp3.apply(check_inclusion2,axis=1)].reset_index(drop=True)

mapping_merge_df_final_grp_temp3['세부사업명_키워드2'] = mapping_merge_df_final_grp_temp3['세부사업명_키워드2'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
mapping_merge_df_final_grp_temp3['납품요구건명_키워드2'] = mapping_merge_df_final_grp_temp3['납품요구건명_키워드2'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

In [49]:
# 필터링 데이터셋 병합
mapping_merge_df_final_grp_fin = pd.concat([mapping_merge_df_final_grp_temp1,mapping_merge_df_final_grp_temp2,mapping_merge_df_final_grp_temp3]).drop_duplicates().reset_index(drop=True)

In [50]:
# 모든 키워드 합 중복 제거
kwd_columns = ['all_kwd','세부사업명_키워드1', '납품요구건명_키워드1', '세부사업명_지역', '납품요구건명_지역', '세부사업명_키워드2', '납품요구건명_키워드2']
mapping_merge_df_final_grp_fin['all_kwd_fin'] = mapping_merge_df_final_grp_fin[kwd_columns].apply(lambda x : (' ').join(x.dropna()),axis=1)
mapping_merge_df_final_grp_fin['all_kwd_fin'] = mapping_merge_df_final_grp_fin['all_kwd_fin'].str.strip()
mapping_merge_df_final_grp_fin['all_kwd_fin'] = mapping_merge_df_final_grp_fin['all_kwd_fin'].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
# mapping_merge_df_final_grp_fin.drop(columns=kwd_columns, inplace
# =True)

mapping_merge_df_final_grp_fin = mapping_merge_df_final_grp_fin.drop_duplicates(['지역명', '자치단체명', '세부사업코드', '세부사업명', '세부사업명_re', '수요기관명', '수요기관지역명',
                                                                                 '납품요구건명', '납품요구건명_re', '수요기관지역명_split1', '자치단체명_re', '수요기관지역명_split2', 'all_kwd_fin']).reset_index(drop=True)

In [51]:
# 최종필터링1 - 지역 일치
mapping_merge_df_final_grp_fin_temp1 = mapping_merge_df_final_grp_fin[(mapping_merge_df_final_grp_fin['세부사업명_지역']!='') & (mapping_merge_df_final_grp_fin['납품요구건명_지역']!='')]
mapping_merge_df_final_grp_fin_temp1 = mapping_merge_df_final_grp_fin_temp1[mapping_merge_df_final_grp_fin_temp1['세부사업명_지역']==mapping_merge_df_final_grp_fin_temp1['납품요구건명_지역']]

In [52]:
# 최종필터링2 - 필수키워드 패턴 포함 & 지역 불일치 제거
mapping_merge_df_final_grp_fin_temp2 = mapping_merge_df_final_grp_fin[(mapping_merge_df_final_grp_fin['세부사업명_키워드2']!='')&(mapping_merge_df_final_grp_fin['납품요구건명_키워드2']!='')].reset_index(drop=True)

# 필요키워드 패턴 포함여부 확인
mapping_merge_df_final_grp_fin_temp2['세부사업명_키워드2'] = mapping_merge_df_final_grp_fin_temp2['세부사업명_키워드2'].str.split() ## 키워드별 정리
mapping_merge_df_final_grp_fin_temp2['납품요구건명_키워드2'] = mapping_merge_df_final_grp_fin_temp2['납품요구건명_키워드2'].str.split() ## 키워드별 정리

mapping_merge_df_final_grp_fin_temp2 = mapping_merge_df_final_grp_fin_temp2[mapping_merge_df_final_grp_fin_temp2.apply(check_inclusion2,axis=1)].reset_index(drop=True)

mapping_merge_df_final_grp_fin_temp2['세부사업명_키워드2'] = mapping_merge_df_final_grp_fin_temp2['세부사업명_키워드2'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
mapping_merge_df_final_grp_fin_temp2['납품요구건명_키워드2'] = mapping_merge_df_final_grp_fin_temp2['납품요구건명_키워드2'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

# 지역 불일치 제거
mapping_merge_df_final_grp_fin_temp2 = mapping_merge_df_final_grp_fin_temp2[mapping_merge_df_final_grp_fin_temp2['세부사업명_지역']==mapping_merge_df_final_grp_fin_temp2['납품요구건명_지역']]

In [53]:
# 최종 필터링 데이터셋
mapping_merge_df_final_grp_final = pd.concat([mapping_merge_df_final_grp_fin_temp1,mapping_merge_df_final_grp_fin_temp2]).drop_duplicates().reset_index(drop=True)
mapping_merge_df_final_grp_final.drop(columns = 'all_kwd', inplace=True)

In [54]:
mapping_merge_df_final_grp_final.to_csv('C:\py_src\midoproject\data/filtered_mapping_df.csv', encoding='utf-8-sig')

In [55]:
# 종합쇼핑몰 기준 필터링
bid_cd1 = list(budget_df_today[budget_df_today['세부사업코드'].isin(mapping_merge_df_final_grp_final['세부사업코드'])]['세부사업코드'])
bid_cd2 = list(budget_df_today[budget_df_today['세부사업명'].isin(mapping_merge_df_final_grp_final['세부사업명'])]['세부사업코드'])

In [56]:
budget_df_today_fin = budget_df_today[~budget_df_today['세부사업코드'].isin(set(bid_cd1 + bid_cd2))].drop_duplicates().reset_index(drop=True)

In [57]:
# 중요 사업 체크
budget_df_today_final = pd.concat([budget_df_today_fin[budget_df_today_fin['세부사업명'].str.contains('인조잔디')],
                                   budget_df_today_fin[~budget_df_today_fin['세부사업명'].str.contains('인조잔디')]],axis=0).reset_index(drop=True)

In [58]:
# 최종데이터셋
budget_df_today_final = budget_df_today_final[['회계연도', '집행일자', '지역명', '자치단체명', '회계구분명', '세부사업명','예산현액', '지출액', '편성액', '분야명', '부문명']].drop_duplicates().reset_index(drop=True)
budget_df_delete = budget_df_delete[['회계연도', '집행일자', '지역명', '자치단체명', '회계구분명', '세부사업명','예산현액', '지출액', '편성액', '분야명', '부문명']].drop_duplicates().reset_index(drop=True)
budget_df_new = budget_df_new[['회계연도', '집행일자', '지역명', '자치단체명', '회계구분명', '세부사업명','예산현액', '지출액', '편성액', '분야명', '부문명']].drop_duplicates().reset_index(drop=True)

In [61]:
# 빅쿼리에 적재
save_dataframe_to_bigquery(budget_df_today_final,'budget','budget_df_listup',bigquerykey_path)
save_dataframe_to_bigquery(budget_df_delete,'budget','budget_df_delete',bigquerykey_path)
save_dataframe_to_bigquery(budget_df_new,'budget','budget_df_new',bigquerykey_path)

Data inserted into table budget_df_listup successfully.
Data inserted into table budget_df_delete successfully.
Data inserted into table budget_df_new successfully.
